<a href="https://colab.research.google.com/github/Thulasi1206/MovieReccomendationSystem/blob/main/Copy_of_Untitled1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import zipfile
import os

# Download the dataset if it doesn't exist
if not os.path.exists("ml-latest-small.zip"):
    !wget http://files.grouplens.org/datasets/movielens/ml-latest-small.zip
    print("Dataset downloaded successfully!")

# Extract the dataset
with zipfile.ZipFile("ml-latest-small.zip",'r') as zip_ref:
    zip_ref.extractall("sample_data")


--2025-02-22 04:10:39--  http://files.grouplens.org/datasets/movielens/ml-latest-small.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 978202 (955K) [application/zip]
Saving to: ‘ml-latest-small.zip’

ml-latest-small.zip 100%[===================>] 955.28K  4.32MB/s    in 0.2s    

2025-02-22 04:10:40 (4.32 MB/s) - ‘ml-latest-small.zip’ saved [978202/978202]

Dataset downloaded successfully!


In [ ]:
import zipfile
with zipfile.ZipFile("ml-latest-small.zip",'r') as zip_ref:
  zip_ref.extractall("sample_data")

In [ ]:
import numpy as np
import pandas as pd


In [ ]:
ratings=pd.read_csv("sample_data/ml-latest-small/ratings.csv")
movies=pd.read_csv("sample_data/ml-latest-small/movies.csv")
movies=movies[['movieId','title']]

In [ ]:
print(ratings.head())

   userId  movieId  rating  timestamp
0       1        1     4.0  964982703
1       1        3     4.0  964981247
2       1        6     4.0  964982224
3       1       47     5.0  964983815
4       1       50     5.0  964982931


In [ ]:
print(movies.head())

   movieId                               title
0        1                    Toy Story (1995)
1        2                      Jumanji (1995)
2        3             Grumpier Old Men (1995)
3        4            Waiting to Exhale (1995)
4        5  Father of the Bride Part II (1995)


In [ ]:
ratings=pd.merge(ratings,movies,on="movieId")

In [ ]:
print(ratings.head())

   userId  movieId  rating  timestamp                      title_x  \
0       1        1     4.0  964982703             Toy Story (1995)   
1       1        3     4.0  964981247      Grumpier Old Men (1995)   
2       1        6     4.0  964982224                  Heat (1995)   
3       1       47     5.0  964983815  Seven (a.k.a. Se7en) (1995)   
4       1       50     5.0  964982931   Usual Suspects, The (1995)   

                       title_y  
0             Toy Story (1995)  
1      Grumpier Old Men (1995)  
2                  Heat (1995)  
3  Seven (a.k.a. Se7en) (1995)  
4   Usual Suspects, The (1995)  


In [ ]:
user_movie_matrix1=ratings.pivot_table(index='userId',columns='title',values='rating').fillna(0)

user_movie_matrix=user_movie_matrix1.values


KeyError: 'title'

In [ ]:
ratings=pd.read_csv("sample_data/ml-latest-small/ratings.csv")
movies=pd.read_csv("sample_data/ml-latest-small/movies.csv")
movies=movies[['movieId','title']]
# Merge ratings and movies DataFrames on 'movieId' to bring 'title' column into ratings
ratings = pd.merge(ratings, movies, on='movieId', how='left') # Ensure 'title' is in ratings
user_movie_matrix1=ratings.pivot_table(index='userId',columns='title',values='rating').fillna(0)

user_movie_matrix=user_movie_matrix1.values

In [ ]:
user_movie_matrix1=ratings.pivot_table(index='userId',columns='title',values='rating').fillna(0)

user_movie_matrix=user_movie_matrix1.values


In [ ]:
!pip install scikit_learn

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train_data,test_data=train_test_split(ratings,test_size=0.2,random_state=42)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
user_similarity=cosine_similarity(user_movie_matrix)
user_similarity_df=pd.DataFrame(user_similarity,index=user_movie_matrix1.index,columns=user_movie_matrix1.index)
print(user_similarity_df.head())

userId       1         2         3         4         5         6         7    \
userId                                                                         
1       1.000000  0.027283  0.059720  0.194395  0.129080  0.128152  0.158744   
2       0.027283  1.000000  0.000000  0.003726  0.016614  0.025333  0.027585   
3       0.059720  0.000000  1.000000  0.002251  0.005020  0.003936  0.000000   
4       0.194395  0.003726  0.002251  1.000000  0.128659  0.088491  0.115120   
5       0.129080  0.016614  0.005020  0.128659  1.000000  0.300349  0.108342   

userId       8         9         10   ...       601       602       603  \
userId                                ...                                 
1       0.136968  0.064263  0.016875  ...  0.080554  0.164455  0.221486   
2       0.027257  0.000000  0.067445  ...  0.202671  0.016866  0.011997   
3       0.004941  0.000000  0.000000  ...  0.005048  0.004892  0.024992   
4       0.062969  0.011361  0.031163  ...  0.085938  0.128273  0

In [ ]:
def predictions(user_movie_matrix,user_similarity_matrix):
  mean_rating=user_movie_matrix.mean(axis=1)
  rating_diff=(user_movie_matrix - mean_rating[:,np.newaxis])
  pred=mean_rating[:,np.newaxis]+user_similarity_matrix.dot(rating_diff)/np.array([np.abs(user_similarity_matrix).sum(axis=1)]).T
  return pred

In [ ]:
predicted_ratings=predictions(user_movie_matrix1.values,user_similarity_df.values)
print(predicted_ratings)

[[ 0.02560548  0.03059038  0.0315963  ...  0.03885832  0.30124734
   0.02144231]
 [-0.04885027 -0.05242888 -0.05357387 ... -0.03084344  0.09906232
  -0.06158879]
 [-0.06432713 -0.06671188 -0.02829163 ... -0.04840215  0.23374791
  -0.07369782]
 ...
 [ 0.18653201  0.18577795  0.18737016 ...  0.20505716  0.40884919
   0.17648002]
 [-0.03611711 -0.03243625 -0.03245831 ... -0.02666406  0.07078844
  -0.03838186]
 [ 0.44031001  0.4056817   0.40707942 ...  0.44138402  0.6051471
   0.39356425]]


In [ ]:
def recommend_movies(user_id,user_movie_matrix,predicted_ratings,movies,num_recommendations=5):
  user_ratings=user_movie_matrix.loc[user_id]
  unratedMovies=user_ratings[user_ratings==0].index
  predicted_scores=pd.Series(predicted_ratings[user_id-1],index=user_movie_matrix.columns)
  recommendations=predicted_scores.loc[unratedMovies].sort_values(ascending=False).head(num_recommendations)
  return recommendations


In [ ]:
print(recommend_movies(1,user_movie_matrix1,predicted_ratings,movies))

title
Shawshank Redemption, The (1994)                             2.640944
Terminator 2: Judgment Day (1991)                            2.080452
Godfather, The (1972)                                        1.855447
Sixth Sense, The (1999)                                      1.661853
Lord of the Rings: The Fellowship of the Ring, The (2001)    1.623584
dtype: float64


In [ ]:
from sklearn.metrics import mean_squared_error
def evaluate_model(user_movie_matrix1,predicted_ratings):
  ratedmovies=user_movie_matrix1.values
  rmse=np.sqrt(mean_squared_error(ratedmovies[ratedmovies.nonzero()],predicted_ratings[ratedmovies.nonzero()]))
  return rmse


print(evaluate_model(user_movie_matrix1,predicted_ratings))


2.965969850360928


In [ ]:
from scipy.sparse.linalg import svds

In [ ]:
u,sigma,vt=svds(user_movie_matrix1.values,k=50)
sigma=np.diag(sigma)

In [ ]:
predicted_ratings_svd=np.dot(np.dot(u,sigma),vt)
predicted_ratings_svd_df=pd.DataFrame(predicted_ratings_svd,index=user_movie_matrix1.index,columns=user_movie_matrix1.columns)
print(predicted_ratings_svd_df)

title   '71 (2014)  'Hellboy': The Seeds of Creation (2004)  \
userId                                                        
1        -0.069287                                 0.018767   
2        -0.028678                                -0.012015   
3         0.017487                                 0.001239   
4        -0.009402                                 0.007128   
5         0.014022                                 0.004154   
...            ...                                      ...   
606       0.023647                                -0.006764   
607       0.009502                                 0.015175   
608       0.001271                                -0.081955   
609      -0.002226                                 0.005341   
610       3.879142                                -0.024543   

title   'Round Midnight (1986)  'Salem's Lot (2004)  \
userId                                                
1                    -0.042895            -0.038633   
2              

In [ ]:
print(recommend_movies(1,user_movie_matrix1,predicted_ratings_svd,movies))

title
Die Hard (1988)                   4.008569
Godfather: Part II, The (1974)    3.304911
Jaws (1975)                       3.302400
Breakfast Club, The (1985)        2.869948
Godfather, The (1972)             2.862019
dtype: float64


In [ ]:
print(evaluate_model(user_movie_matrix1,predicted_ratings_svd))

1.9976811729543087


# Conclusion:
By compared to cosine similarity model ,the error rate is less.So,SVD model give more accurate results.